In [15]:
import numpy as np
import pandas as pd
from glob import glob
import random

In [16]:
files = glob('image/*/*.jpg')
print(files[0])
print(len(files))

image\andrew\andrew1.jpg
205


In [ ]:
random.seed(1)
positive_pairs = []
while len(positive_pairs)<10000:
    j = random.randint(0,204)
    k = random.randint(0,204)
    if files[j].split('\\')[1] == files[k].split('\\')[1]:
        positive_pairs.append(files[j])
        positive_pairs.append(files[k])
print(len(positive_pairs))
positive_pairs = np.array(positive_pairs)
positive_pairs = positive_pairs.reshape((5000,2))
positive_pairs = pd.DataFrame(positive_pairs)
positive_pairs.to_csv('positive_txt.txt',header=None,index=None)

In [ ]:
random.seed(2)
negative_pairs = []
while len(negative_pairs)<10000:
    j = random.randint(0,204)
    k = random.randint(0,204)
    if files[j].split('\\')[1] != files[k].split('\\')[1]:
        negative_pairs.append(files[j])
        negative_pairs.append(files[k])
print(len(negative_pairs))
negative_pairs = np.array(negative_pairs)
negative_pairs = negative_pairs.reshape((5000,2))
negative_pairs = pd.DataFrame(negative_pairs)
negative_pairs.to_csv('negative_txt.txt',header=None,index=None)

In [1]:
from keras.models import Model,Sequential
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
#from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
#from numpy import genfromtxt
import pandas as pd
import tensorflow as tf

#from keras.regularizers import l2
from keras.optimizers import SGD,Adam,RMSprop
from keras.losses import binary_crossentropy
import numpy.random as rng

import matplotlib.pyplot as plt
from sklearn.utils import shuffle


%matplotlib inline
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [3]:
from PIL import Image

dev_number = 8000
batch_size = 128

negative_pairs_file = open('negative_txt.txt', 'r')
negative_pairs_lines = negative_pairs_file.readlines()
positive_pairs_file = open('positive_txt.txt', 'r')
positive_pairs_lines = positive_pairs_file.readlines()

left_image_path_list = []
right_image_path_list = []
label_list = []

for line in negative_pairs_lines:
    left_right = line.split(',')
    left_image_path_list.append(left_right[0])
    right_image_path_list.append(left_right[1].strip('\n'))
    label_list.append(0)

for line in positive_pairs_lines:
    left_right = line.split(',')
    left_image_path_list.append(left_right[0])
    right_image_path_list.append(left_right[1].strip('\n'))
    label_list.append(1)

left_image_path_list = np.asarray(left_image_path_list)
right_image_path_list = np.asarray(right_image_path_list)
label_list = np.asarray(label_list)

np.random.seed(1)
shuffle_index = np.random.permutation(np.arange(len(label_list)))
left_shuffled = left_image_path_list[shuffle_index]
right_shuffled = right_image_path_list[shuffle_index]
label_shuffled = label_list[shuffle_index]

left_train, left_dev = left_shuffled[:dev_number], left_shuffled[dev_number:]
right_train, right_dev = right_shuffled[:dev_number], right_shuffled[dev_number:]
label_train, label_dev = label_shuffled[:dev_number], label_shuffled[dev_number:]

In [4]:
num_pix = 72
def vectorize_imgs(img_path_list):
    image_arr_list = []
    for img_path in img_path_list:
        img = Image.open(img_path)
        img = img.resize((num_pix,num_pix))
        img_arr = np.asarray(img, dtype='float32')
        image_arr_list.append(img_arr)
        
    return image_arr_list


def get_image_array(left_image,right_image,label):
    left = np.asarray(vectorize_imgs(left_image)) / 255.
    right = np.asarray(vectorize_imgs(right_image)) / 255.
    label = np.asarray(label)[:, np.newaxis]
    return left,right,label

In [5]:
left_arr, right_arr, label_arr = get_image_array(left_train, right_train, label_train)

In [6]:
left_dev_arr, right_dev_arr, label_dev_arr = get_image_array(left_dev, right_dev, label_dev)

In [7]:
input_shape = (72, 72, 3)
left_input = Input(input_shape)
right_input = Input(input_shape)
siamese_net = Sequential()
siamese_net.add(Conv2D(32,(5,5),activation='relu',input_shape=input_shape))
siamese_net.add(MaxPooling2D())
siamese_net.add(Conv2D(64,(3,3),activation='relu'))
siamese_net.add(MaxPooling2D())
siamese_net.add(Conv2D(128,(3,3),activation='relu'))
siamese_net.add(MaxPooling2D())
siamese_net.add(Conv2D(256,(1,1),activation='relu'))
siamese_net.add(Flatten())
siamese_net.add(Dense(128,activation="sigmoid"))

encoded_l = siamese_net(left_input)
encoded_r = siamese_net(right_input)
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid')(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
optimizer = Adam(0.00006)
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=['acc'])
siamese_net.count_params()

1733697

In [8]:
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.callbacks import ModelCheckpoint

batch_size = 64
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
earlystop = EarlyStopping(patience=10)
modelsave = ModelCheckpoint(
    filepath='model.h5', save_best_only=True, verbose=1)
siamese_net.fit(
    [left_arr, right_arr],  label_arr, batch_size=batch_size,
    epochs=50, 
    validation_data=([left_dev_arr, right_dev_arr], label_dev_arr),
    callbacks=[annealer, earlystop, modelsave])

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5253 - acc: 0.7378 - val_loss: 0.3728 - val_acc: 0.8580

Epoch 00001: val_loss improved from inf to 0.37283, saving model to model.h5
Epoch 2/50
8000/8000 [==============================] - 4s 488us/step - loss: 0.2241 - acc: 0.9539 - val_loss: 0.1573 - val_acc: 0.9760

Epoch 00002: val_loss improved from 0.37283 to 0.15735, saving model to model.h5
Epoch 3/50
8000/8000 [==============================] - 4s 488us/step - loss: 0.1294 - acc: 0.9774 - val_loss: 0.1167 - val_acc: 0.9805

Epoch 00003: val_loss improved from 0.15735 to 0.11665, saving model to model.h5
Epoch 4/50
8000/8000 [==============================] - 4s 488us/step - loss: 0.0987 - acc: 0.9840 - val_loss: 0.0921 - val_acc: 0.9880

Epoch 00004: val_loss improved from 0.11665 to 0.09209, saving model to model.h5
Epoch 5/50
8000/8000 [==============================] - 4s 486us/step - loss: 0.0745 -

8000/8000 [==============================] - 4s 482us/step - loss: 0.0204 - acc: 0.9993 - val_loss: 0.0342 - val_acc: 0.9935

Epoch 00038: val_loss improved from 0.03445 to 0.03419, saving model to model.h5
Epoch 39/50
8000/8000 [==============================] - 4s 487us/step - loss: 0.0203 - acc: 0.9993 - val_loss: 0.0341 - val_acc: 0.9935

Epoch 00039: val_loss improved from 0.03419 to 0.03409, saving model to model.h5
Epoch 40/50
8000/8000 [==============================] - 4s 489us/step - loss: 0.0202 - acc: 0.9993 - val_loss: 0.0340 - val_acc: 0.9935

Epoch 00040: val_loss improved from 0.03409 to 0.03399, saving model to model.h5
Epoch 41/50
8000/8000 [==============================] - 4s 481us/step - loss: 0.0202 - acc: 0.9993 - val_loss: 0.0339 - val_acc: 0.9935

Epoch 00041: val_loss improved from 0.03399 to 0.03386, saving model to model.h5
Epoch 42/50
8000/8000 [==============================] - 4s 484us/step - loss: 0.0201 - acc: 0.9993 - val_loss: 0.0338 - val_acc: 0.9935

In [9]:
def img_to_encoding(image_path1,image_path2,model):
    img1 = cv2.imread(image_path1)
    img1 = cv2.resize(img1, (72,72))
    img2 = cv2.imread(image_path2)
    img2 = cv2.resize(img2, (72,72))
    x_train1 = np.array([img1])/255
    x_train2 = np.array([img2])/255
    result = model.predict([x_train1,x_train2])
    return result

In [10]:
database = {}
database["loulou"] = "image/loulou/1.jpg"
database["pei"] = "image/pei/pei1.jpg"
database["han"] = "image/han/han21.jpg"
database["andrew"] = "image/andrew/andrew12.jpg"
database["liuyifei"] = "image/liuyifei/fei1.jpg"
database["xidada"] = "image/xidada/xi2.jpg"
database["huge"] = "image/huge/hu1.jpg"   


In [11]:
def who_is_it(test_image_path,database,model):
    max_pro = 0
    dis = []
    for (name, path) in database.items():
        pro = img_to_encoding(test_image_path,path,model)
        dis.append(pro)
        if pro>max_pro:
            max_pro = pro
            identity = name
    if max_pro < 0.2:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(max_pro))
    print(dis)   
    return max_pro, identity,dis

In [14]:
who_is_it("test/2.jpg", database, siamese_net)

it's loulou, the distance is [[ 0.94823778]]
[array([[ 0.94823778]], dtype=float32), array([[ 0.00016784]], dtype=float32), array([[ 0.00044473]], dtype=float32), array([[ 0.12815975]], dtype=float32), array([[  7.64963293e-09]], dtype=float32), array([[  4.93606720e-08]], dtype=float32), array([[  1.75218346e-07]], dtype=float32)]


(array([[ 0.94823778]], dtype=float32),
 'loulou',
 [array([[ 0.94823778]], dtype=float32),
  array([[ 0.00016784]], dtype=float32),
  array([[ 0.00044473]], dtype=float32),
  array([[ 0.12815975]], dtype=float32),
  array([[  7.64963293e-09]], dtype=float32),
  array([[  4.93606720e-08]], dtype=float32),
  array([[  1.75218346e-07]], dtype=float32)])